### Arabidopsis Models

Here we compare Arabidopsis models parameterized for control and cold conditions.

Metabolite constraints for diurnal fluxes for photosynthesis, starch, malate and fumarate are applied as outlined here.
We generate models for Col0 (WT) and the *fum2* mutant. 

In our works submitted for publication "Flux Sampling: A Powerful Tool to Study Metabolism under Changing Environmental Conditions" we have included only the results of the Col0 models. 

In [1]:
# Require packages 
import cobra
from cobra import Model, Reaction, Metabolite
from cobra.flux_analysis import sample
from cobra.flux_analysis import flux_variability_analysis
import numpy as np
import time
from scipy.stats import linregress
import pandas as pd

In [2]:
# Load functions 

def all_constr(model,C,M,F,S):
    # Applying a scaling factor 
    f1=200 
    f2=200
    # Finding model reactions to constrain 
    rMal = model.reactions.get_by_id('Mal_Store')
    rFum = model.reactions.get_by_id('Fum_Store')
    rStarch = model.reactions.get_by_id('Starch_Store')
    rCO2 = model.reactions.get_by_id('Im_CO2')
    FumHA_c = model.reactions.get_by_id('FumHA_c')
    # Metabolite Constraints 
    rCO2.upper_bound = C*1.007
    rCO2.lower_bound = C*0.993
    rMal.upper_bound = M*f2*1.15
    rMal.lower_bound = M*f1*0.85
    rFum.upper_bound = F*f2*1.04
    rFum.lower_bound = F*f1*0.96
    rStarch.upper_bound = S*f2*1.12
    rStarch.lower_bound = S*f1*0.88
    print("Metabolite constraints applied \n")

In [3]:
ANmodel = cobra.io.read_sbml_model("ArabidopsisCoreModel1.xml")

# Changing the directionality of Fum_c to Mal_c to go the other way (day-time model) 
r = ANmodel.reactions.get_by_id('FumHA_c')
r.add_metabolites({ANmodel.metabolites.get_by_id('Fum_c'): +2})
r.add_metabolites({ANmodel.metabolites.get_by_id('H2O_c'): +2})
r.add_metabolites({ANmodel.metabolites.get_by_id('Mal_c'): -2})
print(r.reaction)
print(r.bounds)
print("Directionality changed. Dyson et al. 2016 \n")

# Creating a Malate "Storage" Reactions
r = cobra.Reaction('Mal_Store')
r.name = 'Mal_Store'
r.add_metabolites({ANmodel.metabolites.get_by_id('Mal_c'): -1})
ANmodel.add_reaction(r)
print(r.reaction)

# Creating a Fumarate "Storage" Reactions
r = cobra.Reaction('Fum_Store')
r.name = 'Fum_Store'
r.add_metabolites({ANmodel.metabolites.get_by_id('Fum_c'): -1})
ANmodel.add_reaction(r)
print(r.reaction)

# Creating a Starch "Storage" Reactions
r = cobra.Reaction('Starch_Store')
r.name = 'Starch_Store'
r.add_metabolites({ANmodel.metabolites.get_by_id('starch1_h'): -1})
ANmodel.add_reaction(r)
print(r.reaction)

cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '2*(ATCG00020 and ATCG00680 and ATCG00280 and ATCG00270 and ATCG00580 and ATCG00570 and ATCG00710 and ATCG00080 and ATCG00550 and ATCG00070 and ATCG00560 and ATCG00220 and ATCG00700 and (AT5G66570 or AT3G50820) and AT1G06680 and (AT4G21280 or AT4G05180) and AT1G79040 and AT1G44575 and ATCG00690 and AT3G21055 and AT2G30570 and AT2G06520 and AT1G67740 and ATCG00300)' for <Reaction PSII_h at 0x7f7b63ea8ad0>
cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '2*(ATCG00540 and ATCG00720 and AT4G03280 and ATCG00730 and ATCG00600 and ATCG00590 and AT2G26500 and ATCG00210)' for <Reaction Cytb6f_h at 0x7f7b2dd8de90>
cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '8*(ATCG00490) and 8*(AT5G38430 or AT5G38420 or AT1G67090 or AT5G38410)' for <Reaction RBC_h at 0x7f7b269c1450>
cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '4*(AT3G26650 or AT1G12900) or (2*(AT3G26650 or 

Mal_c --> Fum_c + H2O_c
(0.0, 1000.0)
Directionality changed. Dyson et al. 2016 

Mal_c --> 
Fum_c --> 
starch1_h --> 


In [4]:
rMal = ANmodel.reactions.get_by_id('Mal_Store')
rFum = ANmodel.reactions.get_by_id('Fum_Store')
rStarch = ANmodel.reactions.get_by_id('Starch_Store')
rCO2 = ANmodel.reactions.get_by_id('Im_CO2')
rO2 = ANmodel.reactions.get_by_id('Ex_O2')
FumHA_c = ANmodel.reactions.get_by_id('FumHA_c')
# Applying an objective function for FVA 
# Using FVA only to check constraints not for sampling 
rObj = rCO2
print(rObj.reaction)
ANmodel.objective = rO2
ANmodel.objective_direction = "Maximum"
print(ANmodel.objective)
allreacs = [rMal,rFum,rStarch,rCO2,rO2]

# Col0 Control
all_constr(ANmodel,100.0,0.003,0.006,0.028)
cobra.io.write_sbml_model(ANmodel,"ANmodel2014_Col0Control.xml",use_fbc_package=False)
print(flux_variability_analysis(ANmodel, allreacs))
# Col 0 Cold 
all_constr(ANmodel,100,0.012,0.023,0.051)
cobra.io.write_sbml_model(ANmodel,"ANmodel2014_Col0Cold.xml",use_fbc_package=False)
print(flux_variability_analysis(ANmodel, allreacs))
# Fum2 Control 
all_constr(ANmodel,100,0.012,0.0,0.042)
cobra.io.write_sbml_model(ANmodel,"ANmodel2014_Fum2Control.xml",use_fbc_package=False)
print(flux_variability_analysis(ANmodel, allreacs))
# Fum2 Cold 
all_constr(ANmodel,100,0.025,0.0,0.063)
cobra.io.write_sbml_model(ANmodel,"ANmodel2014_Fum2Cold.xml",use_fbc_package=False)
print(flux_variability_analysis(ANmodel, allreacs))


 --> CO2_c
Maximize
-1.0*Ex_O2_reverse_09189 + 1.0*Ex_O2
Metabolite constraints applied 

              maximum  minimum
Mal_Store       0.690    0.510
Fum_Store       1.248    1.152
Starch_Store    6.272    4.928
Im_CO2        100.700   99.300
Ex_O2         125.000  125.000
Metabolite constraints applied 

                 maximum     minimum
Mal_Store       2.040000    2.040000
Fum_Store       4.416000    4.416000
Starch_Store    8.976000    8.976000
Im_CO2        100.700000  100.700000
Ex_O2         120.510202  120.510202
Metabolite constraints applied 

                 maximum  minimum
Mal_Store       2.760000    2.040
Fum_Store       0.000000    0.000
Starch_Store    8.210582    7.392
Im_CO2        100.700000   99.300
Ex_O2         125.000000  125.000
Metabolite constraints applied 

              maximum  minimum
Mal_Store       4.250    4.250
Fum_Store       0.000    0.000
Starch_Store   11.088   11.088
Im_CO2        100.700  100.700
Ex_O2         112.191  112.191


In [5]:
# Double checking that the constraints were applied. 
ANmodel = cobra.io.read_sbml_model("ANmodel2014_Col0Control.xml")
for r in ANmodel.reactions:
    if max(r.bounds) != 1000.0:
        print(r.reaction)
        print(r.bounds)

cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '2*(ATCG00020 and ATCG00680 and ATCG00280 and ATCG00270 and ATCG00580 and ATCG00570 and ATCG00710 and ATCG00080 and ATCG00550 and ATCG00070 and ATCG00560 and ATCG00220 and ATCG00700 and (AT5G66570 or AT3G50820) and AT1G06680 and (AT4G21280 or AT4G05180) and AT1G79040 and AT1G44575 and ATCG00690 and AT3G21055 and AT2G30570 and AT2G06520 and AT1G67740 and ATCG00300)' for <Reaction PSII_h at 0x7f7b26456750>
cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '2*(ATCG00540 and ATCG00720 and AT4G03280 and ATCG00730 and ATCG00600 and ATCG00590 and AT2G26500 and ATCG00210)' for <Reaction Cytb6f_h at 0x7f7b26456710>
cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '8*(ATCG00490) and 8*(AT5G38430 or AT5G38420 or AT1G67090 or AT5G38410)' for <Reaction RBC_h at 0x7f7b26456890>
cobra/core/reaction.py:394 UserWarning: malformed gene_reaction_rule '4*(AT3G26650 or AT1G12900) or (2*(AT3G26650 or 

 --> CO2_c
(99.3, 100.7)
Mal_c --> 
(0.51, 0.69)
Fum_c --> 
(1.152, 1.248)
starch1_h --> 
(4.928, 6.272)
